<a href="https://colab.research.google.com/github/bkvkrll/-Introduction-to-Natural-Language-Processing/blob/main/lesson12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Урок 12. Модель Transformer-2
Задание

Взять датасет
https://huggingface.co/datasets/merionum/ru_paraphraser
решить задачу парафраза

In [33]:
!pip install transforms

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric

In [37]:
corpus = load_dataset('merionum/ru_paraphraser')

  0%|          | 0/2 [00:00<?, ?it/s]

In [38]:
batch_size = 16

In [39]:
model_name = "IlyaGusev/xlm_roberta_large_headline_cause_simple"

In [40]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
from transformers import AutoTokenizer

In [42]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

https://huggingface.co/IlyaGusev/xlm_roberta_large_headline_cause_simple/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp_stj9ret


storing https://huggingface.co/IlyaGusev/xlm_roberta_large_headline_cause_simple/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/f40a121f6d25907eaaf16bd5c3ac4d4d30f43b98545ee1576d540dd68edbb36d.d2a8e23d6775e1942846b06f703af146248c0bf4b3acda10c7b1b528c13f582c
creating metadata file for /root/.cache/huggingface/transformers/f40a121f6d25907eaaf16bd5c3ac4d4d30f43b98545ee1576d540dd68edbb36d.d2a8e23d6775e1942846b06f703af146248c0bf4b3acda10c7b1b528c13f582c
https://huggingface.co/IlyaGusev/xlm_roberta_large_headline_cause_simple/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp9n868alx


storing https://huggingface.co/IlyaGusev/xlm_roberta_large_headline_cause_simple/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/c5756f9984478ecb47cd24b6c1b937d22d1b9e891ad18acd8ce709e786bacaf1.bcbe2b84be3348c65c2a4860327f92c99e04c740fe7674189304b05b7b5a90d1
creating metadata file for /root/.cache/huggingface/transformers/c5756f9984478ecb47cd24b6c1b937d22d1b9e891ad18acd8ce709e786bacaf1.bcbe2b84be3348c65c2a4860327f92c99e04c740fe7674189304b05b7b5a90d1
https://huggingface.co/IlyaGusev/xlm_roberta_large_headline_cause_simple/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp386xglz7


storing https://huggingface.co/IlyaGusev/xlm_roberta_large_headline_cause_simple/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/209044b3ce6a6c632c48d21d7ad14fa7531afacee452f295f3f2b6322f9c0eb6.a11ebb04664c067c8fe5ef8f8068b0f721263414a26058692f7b2e4ba2a1b342
creating metadata file for /root/.cache/huggingface/transformers/209044b3ce6a6c632c48d21d7ad14fa7531afacee452f295f3f2b6322f9c0eb6.a11ebb04664c067c8fe5ef8f8068b0f721263414a26058692f7b2e4ba2a1b342
loading file https://huggingface.co/IlyaGusev/xlm_roberta_large_headline_cause_simple/resolve/main/sentencepiece.bpe.model from cache at None
loading file https://huggingface.co/IlyaGusev/xlm_roberta_large_headline_cause_simple/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/c5756f9984478ecb47cd24b6c1b937d22d1b9e891ad18acd8ce709e786bacaf1.bcbe2b84be3348c65c2a4860327f92c99e04c740fe7674189304b05b7b5a90d1
loading file https://huggingface.co/IlyaGusev/xlm_roberta_large_headl

In [43]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [44]:
label_list = sorted(set(corpus['train']['class']))
label_list 

['-1', '0', '1']

In [45]:
labels2id = { key:id for id, key in enumerate(label_list)}
labels2id 

{'-1': 0, '0': 1, '1': 2}

In [46]:
id2labels = { id:key for id, key in enumerate(label_list)}
id2labels 

{0: '-1', 1: '0', 2: '1'}

In [47]:
def tokenize_and_align_labels(tokenizer, labels2id):    
    def tokenize_and_align_labels_(examples):
        tokenized_inputs = tokenizer(examples['text_1'],examples['text_2'], truncation=True)
        tokenized_inputs["labels"] = [labels2id[label] for label in examples['class']]
        return tokenized_inputs
    return tokenize_and_align_labels_

In [48]:
tokenized_input = tokenize_and_align_labels(tokenizer, labels2id)(corpus['train'][20:24])
tokenized_input

{'input_ids': [[0, 18449, 1068, 1951, 169066, 59, 42136, 59392, 529, 68056, 49869, 159288, 6, 32345, 14539, 25455, 5, 2, 2, 6, 144048, 145158, 184643, 18449, 28196, 529, 68056, 49869, 159288, 6, 32345, 14539, 25455, 5, 2], [0, 417, 33372, 526, 70847, 132653, 326, 175857, 529, 861, 29034, 7329, 155333, 24863, 5, 2, 2, 417, 33372, 526, 70847, 132653, 326, 175857, 529, 861, 29034, 534, 11591, 149673, 5, 2], [0, 417, 14100, 695, 1200, 49, 30037, 1269, 326, 3699, 108854, 1857, 56869, 90229, 546, 209, 9561, 5, 2, 2, 417, 68642, 69, 53177, 29, 108854, 1857, 47745, 49, 14100, 695, 1200, 90229, 546, 209, 9561, 5, 2], [0, 72085, 138109, 87830, 29, 23262, 2294, 90569, 11974, 24744, 59, 161679, 8568, 5, 2, 2, 69242, 64393, 41976, 4560, 1560, 182076, 29, 29230, 23262, 49, 72085, 138109, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [49]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"][0])
print(tokens)

['<s>', '▁Че', 'ч', 'ня', '▁попросил', 'а', '▁национал', 'истов', '▁со', 'ставить', '▁кодекс', '▁поведения', '▁', 'кав', 'каз', 'цев', '.', '</s>', '</s>', '▁', 'Национал', 'исты', '▁помогут', '▁Че', 'чне', '▁со', 'ставить', '▁кодекс', '▁поведения', '▁', 'кав', 'каз', 'цев', '.', '</s>']


In [50]:
corpus['train'][20]

{'id': '21',
 'id_1': '3',
 'id_2': '4116',
 'text_1': 'Чечня попросила националистов составить кодекс поведения кавказцев.',
 'text_2': 'Националисты помогут Чечне составить кодекс поведения кавказцев.',
 'class': '0'}

In [51]:
tokenized_datasets = corpus.map(tokenize_and_align_labels(tokenizer, labels2id), batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [52]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1924
    })
})

In [53]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [54]:
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification, TrainingArguments, Trainer


model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_list))

model.config.id2label = id2labels 
model.config.label2id = labels2id 

loading configuration file https://huggingface.co/IlyaGusev/xlm_roberta_large_headline_cause_simple/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/577308c326c748161cd1dd0c067991d75cae5a7bc4b4cd64013cd2819a586875.759041bac7e6365837ea8576f87389cc432105d75fd3170f0d098d470e4a748a
Model config XLMRobertaConfig {
  "_name_or_path": "IlyaGusev/xlm_roberta_large_headline_cause_simple",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "not_cause",
    "1": "left_right",
    "2": "right_left"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "left_right": 1,
    "not_cause": 0,
    "right_left": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_

In [55]:
from datasets import load_metric

metric = load_metric("accuracy")

In [56]:
def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)
    
    return metric.compute(predictions=predictions, references=labels)

In [57]:
args = TrainingArguments(
    "paraphras",
    evaluation_strategy = "epoch",
    learning_rate=2e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.05,
    save_strategy='no',
    report_to='none',
)

PyTorch: setting up devices


In [58]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [59]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text_2, id_2, class, id_1, id, text_1. If text_2, id_2, class, id_1, id, text_1 are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 7227
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2260


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.728547,0.656965
2,0.756600,0.700382,0.692827
3,0.588300,0.686240,0.705301
4,0.537800,0.694904,0.713617
5,0.498800,0.729213,0.709459


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text_2, id_2, class, id_1, id, text_1. If text_2, id_2, class, id_1, id, text_1 are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1924
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text_2, id_2, class, id_1, id, text_1. If text_2, id_2, class, id_1, id, text_1 are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1924
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text_2, id_2, class, id_1, id, tex

TrainOutput(global_step=2260, training_loss=0.583625192557816, metrics={'train_runtime': 1621.1197, 'train_samples_per_second': 22.29, 'train_steps_per_second': 1.394, 'total_flos': 3034369670565798.0, 'train_loss': 0.583625192557816, 'epoch': 5.0})

In [60]:
example = tokenized_datasets["test"][40:80]

In [61]:
import torch 

tokens = tokenizer(example['text_1'], example['text_1'], padding=True, truncation=True, return_tensors='pt')
tokens = tokens.to('cuda:0')

with torch.no_grad():
    outputs = model(**tokens)


predicted = outputs.logits.argmax(dim=-1).cpu().numpy()
classes = [id2labels[id_label] for id_label in predicted]

In [62]:
df_example =pd.DataFrame({'text_1':example['text_1'], 'text_2':example['text_2'], 'class':example['class'], 'predict':classes})
df_example

,text_1,text_2,class,predict
0,Владимир Путин освободил от должности почти 20...,Путин снял с должностей более 20 руководителей...,0,1
1,Источник: Якунин озвучил свою месячную зарплат...,Якунин назвал размер своей зарплаты,0,1
2,Apple получила патент на социальную сеть,Apple получила патент на собственную соцсеть,1,1
3,Подвесной потолок обрушился в аэропорту Стамбу...,Четверо пострадали при обрушении подвесного по...,1,1
4,Россия представляет принципиально новый танк,Вашингтон пригрозил России новыми санкциями,-1,1
5,Генсек ООН и премьер Украины обсудили минские ...,Генсек ООН и премьер-министр Украины обсудили ...,1,1
6,Результаты выборов в Великобритании укрепили фунт,Стали известны окончательные результаты выборо...,-1,1
7,Николич: Сербия не говорила о нежелании строит...,Президент Сербии опроверг свои слова об отказе...,0,1
8,СМИ сообщают об обысках в Минкульте по делу о ...,Минкультуры отчиталось перед следствием по дел...,-1,1
9,Любляна отпразднует День Победы вместе с Москвой,Путеводитель по Дню Победы: как провести 9 мая...,-1,1
